# Collect Matches into Flat Files
This notebook collects all of the matches for a certain subset of molecules then writes them to a flat file.

In [1]:
from pymongo import MongoClient
from csv import DictWriter
from pathlib import Path
from tqdm import tqdm

In [2]:
subset = 'random-valid'
output_dir = Path('mentions')

Derived

In [3]:
output_dir.mkdir(exist_ok=True)
output_file = output_dir / f'{subset}-mentions.csv'

## Connect to the Database
We need access to both the molecule record and mention databases

In [4]:
db = MongoClient(port=27856)['cfree']

In [5]:
molecules = db['molecule_record']
print(f'Our database has {molecules.estimated_document_count()} molecules')

Our database has 111995381 molecules


We also need the mentions

In [6]:
mentions = db['mention']
print(f'Our database has {mentions.estimated_document_count()} mentions')

Our database has 22645382 mentions


## Get the IDs of the Molecules that Match
We are going to query all of the IDs to start with

In [7]:
%%time
subset_keys = [x['_id'] for x in molecules.find({'subsets': subset}, projection=['_id'])]
print(f'Total of {len(subset_keys)} molecules in our subset')

Total of 107367 molecules in our subset
CPU times: user 317 ms, sys: 88.1 ms, total: 406 ms
Wall time: 12min 13s


## Write Mentions to Disk
Get all of the mentions and write them to disk

In [8]:
total_matches = 0
matched_keys = set()
with output_file.open('w') as fp:
    # Start writing
    writer = DictWriter(fp, ['filename', 'line', 'name', 'molecule', 'text'])
    writer.writeheader()
    
    # Gather all mentions
    for mention in tqdm(mentions.find({'matches.key': {'$in': subset_keys}})):
        to_write = []
        for match in mention['matches']: 
            if match['key'] in subset_keys:
                matched_keys.add(match['key'])
                total_matches += 1
                to_write.append({
                    'filename': mention['filename'],
                    'line': mention['line'],
                    'text': mention['text'][:-1],
                    'molecule': match['key'],
                    'name': match['name']
                })
        writer.writerows(to_write)

36253it [01:17, 470.09it/s] 


In [9]:
print(f'Printed a total of {total_matches} matches on {len(matched_keys)} molecules')

Printed a total of 37650 matches on 111 molecules
